In [14]:
from bs4 import BeautifulSoup
import re
import string
import os
from collections import Counter,OrderedDict
import nltk
from nltk.tokenize import word_tokenize
import glob
import pickle
import math
import operator
import matplotlib.pyplot as plt

In [3]:
inverted_unigram_dict = dict()
unigram_termcount = {}
unigram_corpus_count = 0
path = 'corpus'
smoothening_factor = 0.35


def casefolding(data):
    return data.lower()

def punctuationHandling(data):
    regex = r"(?<!\d)[.,;:*!\"\'#$%&()+/<=>?@[\]_^`~{|}∑α](?!\d)"
    data = re.sub(regex, "", data, 0)
    data = re.sub(r'\d+', '', data)
    regex = r"[;:*!\"\'#$%&()+/<=>?@[\]_^`~{|}]"
    data = re.sub(regex, "", data, 0)
#     data = data.strip(string.punctuation)
    return data

def removeWhitespace(data):
    data = ' '.join(data.split())
    return data

def createCorpusFile(heading,maincontent):
    fo = open("corpus/"+str(heading)+".txt", "w")
    fo.write(maincontent)
    fo.close()

def parseDocs():
    for filename in glob.glob("test-collection/cacm/*.html"):
        fo = open(filename, "r")
        heading = os.path.basename(filename).split(".")[0]
        data = fo.read()
        maincontent = BeautifulSoup(data, "lxml").text
        maincontent = casefolding(maincontent)
        maincontent = punctuationHandling(maincontent)
        maincontent = removeWhitespace(maincontent)
        createCorpusFile(heading,maincontent)
        
def createIndexDict(file, ngram_dict, inverted_dict):
    for key,value in ngram_dict.items():
        if (inverted_dict.get(key)):
            inverted_dict.get(key).append((os.path.basename(file).split(".")[0],value))
        else:
            inverted_dict[key] = [(os.path.basename(file).split(".")[0],value)]
    return inverted_dict

def getInvertedListCount(index_list):
    invertedlist_count = {}
    for key,val in index_list.items():
        invertedlist_count[key] = [len(val),val]
    return invertedlist_count

def getDocWordFreq(word,document):
    value = 0
    if unigram_invertedlist_count.get(word):
        for val in unigram_invertedlist_count[word][1]:
            if(val[0]==document):
                value = val[1]
    return value

def getCorpusWordFreq(word):
    if unigram_invertedlist_count.get(word):
        return unigram_invertedlist_count[word][0]
    else:
        return 0
    
def query_preprocessor(filepath = 'test-collection/cacm.query.txt'):
    with open(filepath) as f: queries = f.read()
    queries = [l.replace('</DOCNO>', '').replace('\n', ' ').replace('</DOC>', '').replace('<DOC>', '')[1:] for l in queries.split('<DOCNO>')]
    queries = [re.sub(r'^\d*\s\s', '',l) for l in queries]
    queries = [s.lower() for s in queries]
    queries = [punctuationHandling(query) for query in queries]
    queries = [removeWhitespace(query) for query in queries]
    return queries

In [4]:
def writeToFile(queryid,queryname,lmscore_dict,folder_name,system_name):
    fo = open("baseline-runs/"+folder_name+"/"+ "Q" + str(queryid) +".txt", "w")
    for key,val in lmscore_dict.items():
        rank = list(lmscore_dict.keys()).index(key)+1
#         print(queryid,"\tQ0\t",key,"\t",rank,"\t",val,"LMDirichlet\n")
        fo.write(str(queryid)+"\tQ0\t"+str(key)+"\t"+str(rank)+"\t"+str(val)+"\t"+ system_name +"\n")
    fo.close()

In [5]:
def get_bm25(document,query,document_bm25_score_dict):
    total_score = 0
    score = 0
    query_list = query.split(" ")
    
    average_length = unigram_corpus_count / number_of_docs
    for query_word in query_list: 
        R = 0.0
        r = 0.0
        #number of docs containing the term
        
        n = unigram_invertedlist_count.get(query_word)[0] if unigram_invertedlist_count.get(query_word) else 0
#         n = unigram_invertedlist_count.get(query_word)[0]
        
        
        #Total number of documents
        N = number_of_docs
        k1 = 1.2
        k2 = 100
        #freq of the word in query
        qf = query_list.count(query_word)
        
        #freq of word in the doc
        f = getDocWordFreq(query_word,document)
        
        #to be calculated using b= 0.75
        b = 0.75
        K = k1*((1-b) + b*(float(unigram_termcount[document])/float(average_length)))
        smoothening = 0.5
        
        first_part = math.log(((r + 0.5)/(R - r + 0.5))/((n - r + 0.5)/(N - n - R + r + 0.5)))
        second_part = ((k1 + 1)*f)/(K + f)
        third_part = ((k2 + 1) * qf)/(k2 + qf)
        score = first_part*second_part*third_part
        
        total_score += score
    document_bm25_score_dict[document] = total_score
    return document_bm25_score_dict

In [6]:
def get_tf_idf(document,query,document_tfidfscore_dict):
    total_score = 0
    score = 0
    query_list = query.split(" ")
    for query_word in query_list: 
        if(getDocWordFreq(query_word,document)!=0 and unigram_termcount.get(document)):
            tf_value  = float(getDocWordFreq(query_word,document))/float(unigram_termcount[document])
        else:
            tf_value = 0
        if(unigram_invertedlist_count.get(query_word)):
            idf_value = math.log(number_of_docs / unigram_invertedlist_count[query_word][0])
        else:
            idf_value = 0
            
        score = tf_value * idf_value
        total_score += score
    document_tfidfscore_dict[document] = total_score
    return document_tfidfscore_dict

In [7]:
def populate_bm25(queryString):
    document_bm25_score_dict = {}
    docList = []
    query_list = queryString.split(" ")
    
    for query in query_list:
        if unigram_invertedlist_count.get(query):
            query_doclist = unigram_invertedlist_count[query][1]
            for i in query_doclist:
                docList.append(i[0])
        else:
            query_doclist = []
            
    for docid in docList:
        document_bm25_score_dict = get_bm25(docid,queryString,document_bm25_score_dict)
        document_bm25_score_dict = dict(sorted(document_bm25_score_dict.items(), key=operator.itemgetter(1), reverse=True)[:100])
    return document_bm25_score_dict



In [8]:
def populate_tfidf(queryString):
    document_tfidfscore_dict = {}
    docList = []
    query_list = queryString.split(" ")
    
    for query in query_list:
        if unigram_invertedlist_count.get(query):
            query_doclist = unigram_invertedlist_count[query][1]
            for i in query_doclist:
                docList.append(i[0])
        else:
            query_doclist = []
            
    for docid in docList:
        document_tfidfscore_dict = get_tf_idf(docid,queryString,document_tfidfscore_dict)
        document_tfidfscore_dict = dict(sorted(document_tfidfscore_dict.items(), key=operator.itemgetter(1), reverse=True)[:100])
    
    return document_tfidfscore_dict

In [9]:
def getJMQLScores(document,query,document_lmscore_dict):
    total_score = 0
    score = 0
    query_list = query.split(" ")
    for query_word in query_list: 
        if (getDocWordFreq(query_word,document) != 0) and (getCorpusWordFreq(query_word) != 0) :
            score = math.log(((1 - smoothening_factor) * float(getDocWordFreq(query_word,document) / unigram_termcount[document])) +\
                             (smoothening_factor * float(getCorpusWordFreq(query_word) / unigram_corpus_count)))
            
        else:
            score = 0
        total_score += score
    document_lmscore_dict[document] = total_score
    return document_lmscore_dict

In [10]:
def populateJMQL(queryString):
    document_lmscore_dict = {}
    docList = []
    query_list = queryString.split(" ")
    
    for query in query_list:
        if unigram_invertedlist_count.get(query):
            query_doclist = unigram_invertedlist_count[query][1]
            for i in query_doclist:
                docList.append(i[0])
        else:
            query_doclist = []
            
    for docid in docList:
        document_lmscore_dict = getJMQLScores(docid,queryString,document_lmscore_dict)
        document_lmscore_dict = dict(sorted(document_lmscore_dict.items(), key=operator.itemgetter(1), reverse=True)[:100])
    return document_lmscore_dict

In [11]:
# parseDocs()
for filename in glob.glob("corpus/*.txt"):
    fo          = open(filename, "r")
    data        = fo.read()
    tokens      = nltk.word_tokenize(data)
    unigramlist = nltk.word_tokenize(data)
    
    unigram_termcount[os.path.basename(filename).split(".")[0]] = len(unigramlist)
    
    unigram_corpus_count  = unigram_corpus_count + len(unigramlist)
    unigram_dict          = Counter(unigramlist)
    inverted_unigram_dict = createIndexDict(filename, unigram_dict, inverted_unigram_dict)
    fo.close()
    
unigram_invertedlist_count = getInvertedListCount(inverted_unigram_dict)
number_of_docs = len(glob.glob('corpus/*.txt'))

#with open('reusable_data/unigram_invertedlist_count.pkl', 'wb') as f:
#    pickle.dump(unigram_invertedlist_count, f)
    
#with open('reusable_data/inverted_index.pkl', 'wb') as f:
#    pickle.dump(inverted_unigram_dict, f)

all_queries = query_preprocessor()[1:]



for i in range(len(all_queries)):
    print(all_queries[i])
    bm25_score_dict = populate_bm25(all_queries[i])
    break
#     writeToFile(i+1,all_queries[i],bm25_score_dict,"task1-bm25","ccisneu_wordunigram_BM25")
#     tfidf_score_dict = populate_tfidf(all_queries[i])
#     writeToFile(i+1,all_queries[i],tfidf_score_dict,"task1-tfidf","ccisneu_wordunigram_TFIDF")
#     jmqlscore_dict = populateJMQL(all_queries[i])
#     writeToFile(i+1,all_queries[i],jmqlscore_dict,"task1-JMQL","ccisneu_wordunigram_JMQL")


what articles exist which deal with tss time sharing system an operating system for ibm computers


In [13]:
unigram_invertedlist_count['mea']

{'measurements': [16,
  [('CACM-2095', 2),
   ('CACM-1601', 2),
   ('CACM-2450', 1),
   ('CACM-2859', 1),
   ('CACM-2669', 1),
   ('CACM-0973', 2),
   ('CACM-0963', 1),
   ('CACM-2998', 2),
   ('CACM-1443', 1),
   ('CACM-2016', 1),
   ('CACM-3052', 1),
   ('CACM-1108', 1),
   ('CACM-1810', 1),
   ('CACM-3020', 1),
   ('CACM-2892', 2),
   ('CACM-2891', 1)]],
 'of': [2079,
  [('CACM-2095', 8),
   ('CACM-0696', 2),
   ('CACM-1588', 5),
   ('CACM-2081', 6),
   ('CACM-2903', 5),
   ('CACM-2730', 1),
   ('CACM-2042', 3),
   ('CACM-0655', 5),
   ('CACM-0133', 1),
   ('CACM-1563', 2),
   ('CACM-1577', 1),
   ('CACM-0669', 1),
   ('CACM-2718', 5),
   ('CACM-1211', 2),
   ('CACM-1946', 8),
   ('CACM-0480', 1),
   ('CACM-2297', 12),
   ('CACM-2283', 5),
   ('CACM-0494', 1),
   ('CACM-2254', 9),
   ('CACM-0325', 1),
   ('CACM-2526', 6),
   ('CACM-1991', 3),
   ('CACM-1749', 6),
   ('CACM-2268', 4),
   ('CACM-3176', 3),
   ('CACM-1007', 3),
   ('CACM-0319', 6),
   ('CACM-1013', 3),
   ('CACM-3162',

In [12]:
bm25_score_dict

{'CACM-0098': 13.927814221046761,
 'CACM-0202': 13.595948948372854,
 'CACM-0252': 15.474027865302013,
 'CACM-0414': 16.24043175017192,
 'CACM-0585': 17.133244231049094,
 'CACM-0637': 15.613456796723018,
 'CACM-0698': 15.495907032443249,
 'CACM-0971': 15.38536467519953,
 'CACM-1033': 19.524583605917798,
 'CACM-1046': 14.68151193071902,
 'CACM-1161': 20.328663802693665,
 'CACM-1163': 14.140202294301396,
 'CACM-1168': 17.34461292922429,
 'CACM-1173': 15.061270576691106,
 'CACM-1179': 15.34694014894371,
 'CACM-1195': 14.902680880628589,
 'CACM-1225': 15.788521011991302,
 'CACM-1226': 14.500766331460921,
 'CACM-1236': 14.861434860959275,
 'CACM-1264': 17.306688009751586,
 'CACM-1304': 15.431094026203052,
 'CACM-1315': 15.267465124485229,
 'CACM-1393': 16.394762784634423,
 'CACM-1410': 20.184611800947618,
 'CACM-1417': 13.42663734080654,
 'CACM-1440': 17.193127408760713,
 'CACM-1462': 14.63212261705236,
 'CACM-1464': 17.39248082458295,
 'CACM-1472': 14.388221903663462,
 'CACM-1482': 14.40943